---
# --- Day 21: Monkey Math ---
---

In [130]:
import numpy as np
import re
from numpy.polynomial import Polynomial
from typing import List, Dict

## Load data

In [131]:
full_puzzle_data = True

In [132]:
file_suffix = "" if full_puzzle_data else "_test"
with open(f"data/day21_input{file_suffix}.txt", "r") as f:
    data = f.read().splitlines()

In [97]:
def initiate_data(data: List[str]) -> (Dict, Dict):
    monkeys = {}
    dependency = {}
    for row in data:
        mname, oper = row.split(": ")
        if re.match("[0-9]+", oper):
            record = {
                "left": None,
                "right": None,
                "sign": None,
                "number": int(oper)
            }
        else:
            left, sign, right  = oper.split(" ")
            record = {
                "left": left,
                "right": right,
                "sign": sign,
                "number": None
            }
            dependency[left] = dependency.get(left, []) + [mname]
            dependency[right] = dependency.get(left, []) + [mname]
        monkeys[mname] = record
    return monkeys, dependency

In [98]:
monkeys, dependency = initiate_data(data)

## --- Part One ---

In [99]:
def check_number_and_propagate(name: str, monkeys: dict, dependency: dict):
    if monkeys[name]["number"] is None:
        left_number = monkeys[monkeys[name]["left"]]["number"]
        right_number = monkeys[monkeys[name]["right"]]["number"]
        if (left_number is not None) and (right_number is not None):
            sign = monkeys[name]["sign"]
            opr = f"{left_number}{sign}{right_number}"
            res = eval(opr)
            monkeys[name]["number"] = int(res)
            if name in dependency:
                for tgt in dependency[name]:
                    check_number_and_propagate(tgt, monkeys, dependency)

In [100]:
for rnd in range(5):
    print(f"*** Round {rnd+1} ***")    
    for m in monkeys.keys():
        check_number_and_propagate(m, monkeys, dependency)
    root_number = monkeys["root"]["number"]
    if root_number is not None:
        print(f"The number yelled by the monkey 'root' is {root_number}")
        break

*** Round 1 ***
The number yelled by the monkey 'root' is 152


## --- Part Two ---

In [139]:
monkeys, dependency = initiate_data(data)

In [140]:
monkeys["root"]["sign"] = "-"
monkeys["humn"]["number"] = Polynomial([0, 1])

In [144]:
def check_poly_and_propagate(name: str, monkeys: dict, dependency: dict):
    if monkeys[name]["number"] is None:
        left_number = monkeys[monkeys[name]["left"]]["number"]
        right_number = monkeys[monkeys[name]["right"]]["number"]
        if (left_number is not None) and (right_number is not None):
            sign = monkeys[name]["sign"]
            if sign == "-":
                res = left_number - right_number
            elif sign == "+":
                res = left_number + right_number
            elif sign == "*":
                res = left_number * right_number
            else:
                res = left_number / right_number
            if isinstance(res, Polynomial):
                if res.degree() == 0:
                    res = res.coef[0]
            else:
                res = res
            monkeys[name]["number"] = res
            if name in dependency:
                for tgt in dependency[name]:
                    check_poly_and_propagate(tgt, monkeys, dependency)

In [145]:
for rnd in range(5):
    print(f"*** Round {rnd+1} ***")    
    for m in monkeys.keys():
        check_poly_and_propagate(m, monkeys, dependency)
    root_number = monkeys["root"]["number"]
    if root_number is not None:
        print("Expression for the monkey 'root' found.")
        break

*** Round 1 ***
Expression for the monkey 'root' found.


In [148]:
root_number = int(np.round(monkeys["root"]["number"].roots()[0]))
print(f"The number that should be yelled by me is {root_number}")

The number that should be yelled by me is 3352886133831
